## Основы анализа данных

## Домашнее задание 6

### О задании

1) Возьмите три количественных признака в Ваших данных и сформируйте бинарный признак по каждому из них. (Один из этих трех бинарных признаков может быть взят из данных непосредственно, тогда надо будет бинаризовать только два количественных признака.)

2) Один из трех признаков сделайте «выходным» и рассмотрите две таблицы сопряженности для предсказания значения выходного признака по каждому из входных. Рассчитайте характеристики аккуратности, точности и полноты для каждой из таблиц. Откомментируйте и сравните результаты.

Для начала подключим ряд библиотек, которые мы будем использовать впоследствии.

In [191]:
import pandas as pd
import numpy as np

Загрузим данные

In [192]:
practice_data = pd.read_csv('../../data/Vehicles0515.csv')
practice_data = practice_data.replace(-1, np.NaN)
practice_data = practice_data.dropna(subset=['Age_of_Driver', 'Driving_Experience', 'Sex_of_Driver'])
practice_data.head()

,Unnamed: 0,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,...,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Driving_Experience,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
0,0,200501BS00001,1,9,0.0,18,0,0.0,0.0,0.0,...,1.0,15.0,2,94.0,10.0,NaN,NaN,NaN,7.0,1.0
1,1,200501BS00002,1,11,0.0,4,0,3.0,0.0,0.0,...,1.0,1.0,1,62.0,7.0,8268.0,2.0,3.0,NaN,NaN
2,2,200501BS00003,1,11,0.0,17,0,0.0,0.0,4.0,...,1.0,1.0,1,55.0,6.0,8300.0,2.0,5.0,2.0,1.0
3,3,200501BS00003,2,9,0.0,2,0,0.0,0.0,0.0,...,1.0,15.0,1,82.0,9.0,1762.0,1.0,6.0,1.0,1.0
4,4,200501BS00004,1,9,0.0,18,0,0.0,0.0,0.0,...,1.0,15.0,2,69.0,8.0,1769.0,1.0,4.0,2.0,1.0


Сформируем из количественных признаков "Возраст водителя" и "Стаж водителя" бинарные признаки, а также примем во внимание признак "Пол водителя".

Бинаризуем признаки. Для этого сначала найдем средние значения каждого из них.

In [193]:
mean_age = round(practice_data['Age_of_Driver'].mean())
mean_experience = round(practice_data['Driving_Experience'].mean())
print("Средний по выборке возраст водителя: %d, средний по выборке стаж вождения: %d" % (mean_age, mean_experience))

Средний по выборке возраст водителя: 57, средний по выборке стаж вождения: 7


Затем создадим новые столбцы так, что <b>True</b> эквивалентно тому, что значение признака больше или равно среднему по столбцу, и <b>False</b> тому, что меньше.

In [194]:
final_data = pd.DataFrame()

In [195]:
final_data['Age_of_Driver_binary'] = practice_data.apply(lambda row: row['Age_of_Driver'] >= mean_age,axis=1)
final_data['Driving_Experience_binary'] = practice_data.apply(lambda row: row['Driving_Experience'] >= mean_experience,axis=1)

Переведем признак "Пол водителя" в бинарный.

In [196]:
final_data['Sex_of_Driver_binary'] = (practice_data['Sex_of_Driver'] - 1).astype("bool")

Таким образом, получена таблица "новых" значений.

In [204]:
final_data.head()

,Age_of_Driver_binary,Driving_Experience_binary,Sex_of_Driver_binary
0,True,True,True
1,True,True,False
2,False,False,False
3,True,True,False
4,True,True,True


Рассмотрим таблицу сопряженности такую, что "Пол водителя" - входной признак и "Стаж водителя" - выходной.

In [205]:
sex_exp_table = pd.crosstab(final_data['Sex_of_Driver_binary'], final_data['Driving_Experience_binary'])
sex_exp_table.head()

Driving_Experience_binary,False,True
Sex_of_Driver_binary,,
False,14503,14719
True,4981,5006


Посчитаем <b>accuracy</b>, <b>precision</b> и <b>recall</b> в этой таблице.

In [206]:
sex_exp_accuracy = (sex_exp_table[0][0] + sex_exp_table[1][1]) / sex_exp_table.values.sum()
sex_exp_precision = sex_exp_table[1][1] / (sex_exp_table[0][1] + sex_exp_table[1][1])
sex_exp_recall = sex_exp_table[1][1] / (sex_exp_table[1][0] + sex_exp_table[1][1])

Таким образом, получены характеристики для таблицы "Пол водителя" / "Стаж водителя".

In [200]:
print("Аккуратность: {0:.3f}\nТочность: {1:.3f}\nПолнота: {2:.3f}".format(sex_exp_accuracy, sex_exp_precision, sex_exp_recall))

Аккуратность: 0.498
Точность: 0.501
Полнота: 0.254


Исходя из проделанных вычислений, можно сделать вывод о том, что пол водителя <b>мало коррелирует</b> со стажем вождения.
Однако <b>на данном датасете</b> можно наблюдать следующее: при стаже более 7 лет, вероятность того, что водитель, попавший в дтп, является мужчиной, составляет всего около 25%.

Теперь точно так же рассмотрим таблицу сопряженности такую, что  "Возраст водителя" - входной признак и "Стаж водителя" - выходной.

In [201]:
age_exp_table = pd.crosstab(final_data['Age_of_Driver_binary'], final_data['Driving_Experience_binary'])
age_exp_table.head()

Driving_Experience_binary,False,True
Age_of_Driver_binary,,
False,19484,1076
True,0,18649


Посчитаем <b>accuracy</b>, <b>precision</b> и <b>recall</b> в этой таблице.

In [202]:
age_exp_accuracy = (age_exp_table[0][0] + age_exp_table[1][1]) / age_exp_table.values.sum()
age_exp_precision = age_exp_table[1][1] / (age_exp_table[0][1] + age_exp_table[1][1])
age_exp_recall = age_exp_table[1][1] / (age_exp_table[1][0] + age_exp_table[1][1])

 Таким образом, получены характеристики для таблицы "Возраст водителя" / "Стаж водителя".

In [203]:
print("Аккуратность: {0:.3f}\nТочность: {1:.3f}\nПолнота: {2:.3f}".format(age_exp_accuracy, age_exp_precision, age_exp_recall))

Аккуратность: 0.973
Точность: 1.000
Полнота: 0.945


Исходя из проделанных вычислений, можно сделать вывод о том, что возраст водителя <b>сильно коррелирует</b> со стажем вождения (чем больше возраст, тем больше стаж, соответственно).
Также <b>на данном датасете</b> можно наблюдать следующее: если возраст водителя в исходных данных выше 57, с вероятностью 100% его стаж выше 7 лет (значения приближенные). Помимо этого, с вероятностью около 90%, водитель, чей стаж больше 7 лет, старше 57.